In [1]:
import os
import pandas as pd
import seaborn as sns
from Priorizacion_funciones.ocupacion_nodos_funs import *
from Priorizacion_funciones.modelo_forecast_funs import *
from Priorizacion_funciones.analisis_funcional_nodos import *
import skfda
os.getcwd()

'C:\\Users\\DarioAlejandroLatorr\\OneDrive\\claro_20_abril_2021\\ENVIO_DARIO'

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [3]:
## archivos txt: lectura de archivos de agosto de 2020 a enero 2021
path_fun = lambda x: f'Datos_heavy_users/consolidar_trafico_users/ipdr202{x}01.txt'
list_mes = ['008','009','010','011','012','101']
list_cols = ['_c0','flbb_down_traffic_bytes','subscriber_id','flbb_up_traffic_bytes']
df_1 = pd.concat([pd.read_csv(path_fun(x), sep='|', usecols=list_cols) for x in list_mes])

df_1 = df_1[(df_1['flbb_down_traffic_bytes']!='flbb_down_traffic_bytes')].copy()
df_1 = df_1[df_1['subscriber_id']!='Unknown'].copy()

df_1['flbb_down_traffic_bytes'] = df_1['flbb_down_traffic_bytes'].astype('float')
df_1['flbb_up_traffic_bytes'] = df_1['flbb_up_traffic_bytes'].astype('float')
df_1['subscriber_id'] = df_1['subscriber_id'].astype('int64')
df_1['_c0'] = [x[:-2] for x in df_1['_c0']]
dic_names = {'_c0':'ANIO_MES','flbb_down_traffic_bytes':'TRAFICO_DOWN_BYTES',
            'flbb_up_traffic_bytes':'TRAFICO_UP_BYTES', 'subscriber_id':'CUENTA'}
df_1.rename(columns=dic_names, inplace=True)
## archivos csv: lectura de archivos de marzo a mayo 
path_fun = lambda x:  f'Datos_heavy_users/consolidar_trafico_users/traf_user_20210{x}.csv'
list_mes = ['3','4','5']
list_cols = ['ANIO_MES','TRAFICO_DOWN_BYTES','CUENTA','TRAFICO_UP_BYTES']
df_2 = pd.concat([pd.read_csv(path_fun(x), usecols=list_cols) for x in list_mes])

df = pd.concat([df_1, df_2])

df['TRAFICO_TOTAL_BYTES'] = df['TRAFICO_DOWN_BYTES'] + df['TRAFICO_UP_BYTES']

In [4]:
pd.read_csv(path_fun(5), usecols=['SERVICIOS']).value_counts()

SERVICIOS                               
@ + VOZ                                     1742283
TV + @ + VOZ                                1479195
Unknown                                        3129
CR 7                                             15
CL 19                                            14
                                             ...   
CL 56FSUR TORRE 9                                 1
AV LAROSITA TORRE NORTE                           1
CL 57 PIEDRAPINTADA ETAPA 1 MZC                   1
CL 57 TORRE 1                                     1
VIA VARIANTECAJICAZIPAQUIRA KM4 TORRE 11          1
Length: 2608, dtype: int64

In [5]:
df_total = pd.pivot_table(df, columns ='ANIO_MES', values='TRAFICO_TOTAL_BYTES',
                         index ='CUENTA', aggfunc='sum').reset_index()
df_total.rename(columns = {x: str(x) for x in df_total.columns}, inplace=True)
df_total.set_index('CUENTA', inplace=True)
lista = ['202008', '202009', '202010', '202011', '202012', '202101',
         '202103','202104', '202105']
df_total = df_total[lista]
print(f'{100*"-"}\nFaltantes:\n{100*"-"}')
for col in df_total.columns[1:]:
    print(f'{col} --> {sum(df_total[col].isnull())}')
    
df_total.interpolate(method='linear', limit_area='inside',axis=1,inplace=True)

print(f'{100*"-"}\nFaltantes después de interpolar:\n{100*"-"}')
for col in df_total.columns[1:]:
    print(f'{col} --> {sum(df_total[col].isnull())}')

----------------------------------------------------------------------------------------------------
Faltantes:
----------------------------------------------------------------------------------------------------
202009 --> 997499
202010 --> 989830
202011 --> 1023817
202012 --> 982092
202101 --> 1016260
202103 --> 722489
202104 --> 554072
202105 --> 629615
----------------------------------------------------------------------------------------------------
Faltantes después de interpolar:
----------------------------------------------------------------------------------------------------
202009 --> 976124
202010 --> 953103
202011 --> 971749
202012 --> 931239
202101 --> 956025
202103 --> 619221
202104 --> 510754
202105 --> 629615


In [6]:
df_total.to_csv('Datos_heavy_users/consolidar_trafico_users/trafico_total_meses.csv')

## Consolidación

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
df_total = pd.read_csv('Datos_heavy_users/consolidar_trafico_users/trafico_total_meses.csv', index_col=['CUENTA'])

In [4]:
df_total_resumen = df_total[['202105']].copy()

df_total_resumen['TRAFICO MEDIO'] = df_total.mean(1)
df_total_resumen['TRAFICO MINIMO'] = df_total.min(1)
df_total_resumen['TRAFICO MAXIMO'] = df_total.max(1)
df_total_resumen['CONTEO MEDICIONES'] = df_total.drop(columns=['202105']).count(1)
df_total_resumen['DESVEST TRAFICO'] = df_total.std(1)
df_total_resumen['TRAFICO MEDIO AG20-ENE21'] = df_total[['202008', '202009', '202010', '202011', '202012', '202101']].mean(1)
df_total_resumen['TRAFICO MEDIO MAR21-MAY21'] = df_total[['202103','202104', '202105']].mean(1)
df_total_resumen['TRAFICO ULT TRES MESES'] = (df_total[['202103','202104', '202105']].count(1) ==3)

list_cols_hu = []
for col in df_total.drop(columns=['202105']).columns:
    col_name = col+'_HU'
    df_total_resumen[col_name] = (((df_total[col] -df_total[col].mean())/(df_total[col].std())) >1)*1 
    list_cols_hu.append(col_name)
df_total_resumen['PROP HU RESP CONTEO MEDICIONES'] =  df_total_resumen[list_cols_hu].sum(1)/df_total_resumen['CONTEO MEDICIONES']
df_total_resumen.info(show_counts =True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3376013 entries, 307 to 99699993
Data columns (total 18 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   202105                          2746398 non-null  float64
 1   TRAFICO MEDIO                   3376013 non-null  float64
 2   TRAFICO MINIMO                  3376013 non-null  float64
 3   TRAFICO MAXIMO                  3376013 non-null  float64
 4   CONTEO MEDICIONES               3376013 non-null  int64  
 5   DESVEST TRAFICO                 3214683 non-null  float64
 6   TRAFICO MEDIO AG20-ENE21        2635654 non-null  float64
 7   TRAFICO MEDIO MAR21-MAY21       3052388 non-null  float64
 8   TRAFICO ULT TRES MESES          3376013 non-null  bool   
 9   202008_HU                       3376013 non-null  int32  
 10  202009_HU                       3376013 non-null  int32  
 11  202010_HU                       3376013 non-null  int32  
 1

In [5]:
cols_info = ['NODO','CODIGO_CUENTA','RENTA_MENSUAL','ESTRATO'] 
df_total_resumen = df_total_resumen.reset_index().merge(pd.read_csv("DATA_RR/DATOS_RR_202105.csv",sep=';', usecols=cols_info),
                                                       how='right', left_on ='CUENTA', right_on='CODIGO_CUENTA')
df_total_resumen.info(show_counts =True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3549552 entries, 0 to 3549551
Data columns (total 23 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   CUENTA                          3006659 non-null  float64
 1   202105                          2812213 non-null  float64
 2   TRAFICO MEDIO                   3006659 non-null  float64
 3   TRAFICO MINIMO                  3006659 non-null  float64
 4   TRAFICO MAXIMO                  3006659 non-null  float64
 5   CONTEO MEDICIONES               3006659 non-null  float64
 6   DESVEST TRAFICO                 2962769 non-null  float64
 7   TRAFICO MEDIO AG20-ENE21        2313831 non-null  float64
 8   TRAFICO MEDIO MAR21-MAY21       2952168 non-null  float64
 9   TRAFICO ULT TRES MESES          3006659 non-null  object 
 10  202008_HU                       3006659 non-null  float64
 11  202009_HU                       3006659 non-null  float64
 12  

In [6]:
df_total_resumen['HEAVY USERS'] = (df_total_resumen['PROP HU RESP CONTEO MEDICIONES']>=0.5)
df_total_resumen['HEAVY USERS'].value_counts()

False    3417715
True      131837
Name: HEAVY USERS, dtype: int64

In [7]:
sns.boxplot(data=df_total_resumen, x='HEAVY USERS', y='TRAFICO MEDIO')

<AxesSubplot:xlabel='HEAVY USERS', ylabel='TRAFICO MEDIO'>

In [8]:
sns.boxplot(data=df_total_resumen, x='HEAVY USERS', y='TRAFICO MINIMO')

<AxesSubplot:xlabel='HEAVY USERS', ylabel='TRAFICO MINIMO'>

In [9]:
sns.boxplot(data=df_total_resumen, x='HEAVY USERS', y='TRAFICO MAXIMO')

<AxesSubplot:xlabel='HEAVY USERS', ylabel='TRAFICO MAXIMO'>

In [11]:
sns.boxplot(data=df_total_resumen, y='HEAVY USERS', x='202105', orient='h')

<AxesSubplot:xlabel='202105', ylabel='HEAVY USERS'>

In [15]:
print(df_total_resumen.loc[df_total_resumen['HEAVY USERS'],'202105'].sum() /df_total_resumen['202105'].sum())
print(df_total_resumen.loc[df_total_resumen['HEAVY USERS'],'202105'].count() /df_total_resumen['202105'].count())

0.24037851001634797
0.04543290284199668
